# 🛠 Exercises

## 1. Build and fit a model using the same data we have here but with the MobileNetV2 architecture feature extraction ([`mobilenet_v2_100_224/feature_vector`](https://tfhub.dev/google/imagenet/mobilenet_v2_100_224/feature_vector/4)) from TensorFlow Hub, how does it perform compared to our other models?

In [1]:
# Import libraries
import zipfile
import tensorflow as tf
import tensorflow_hub as hub

### Data preprocessing

In [2]:
# Download data
!wget https://storage.googleapis.com/ztm_tf_course/food_vision/10_food_classes_10_percent.zip

# Unzip data
zip_ref = zipfile.ZipFile("10_food_classes_10_percent.zip", "r")
zip_ref.extractall()
zip_ref.close()

--2023-11-20 12:50:27--  https://storage.googleapis.com/ztm_tf_course/food_vision/10_food_classes_10_percent.zip
Resolving storage.googleapis.com (storage.googleapis.com)... 74.125.200.207, 74.125.130.207, 74.125.68.207, ...
Connecting to storage.googleapis.com (storage.googleapis.com)|74.125.200.207|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 168546183 (161M) [application/zip]
Saving to: ‘10_food_classes_10_percent.zip’

10_food_classes_10_ 100%[===================>] 160.74M  19.0MB/s    in 9.1s    

2023-11-20 12:50:37 (17.7 MB/s) - ‘10_food_classes_10_percent.zip’ saved [168546183/168546183]



In [3]:
# Set data paths
train_dir = "10_food_classes_10_percent/train"
test_dir = "10_food_classes_10_percent/test"

# Create generator
train_datagen = tf.keras.preprocessing.image.ImageDataGenerator(rescale = 1 / 255)
test_datagen = tf.keras.preprocessing.image.ImageDataGenerator(rescale = 1 / 255)

# Create data
train_data = train_datagen.flow_from_directory(directory=train_dir,
                                               target_size=(224, 224),
                                               class_mode='categorical',
                                               batch_size=32)

test_data = test_datagen.flow_from_directory(directory=test_dir,
                                             target_size=(224, 224),
                                             class_mode='categorical',
                                             batch_size=32)

Found 750 images belonging to 10 classes.
Found 2500 images belonging to 10 classes.


### Model

In [4]:
# URL for mobilenetv2 feature extraction
mobilenet_v2 = "https://kaggle.com/models/google/mobilenet-v2/frameworks/TensorFlow2/variations/100-224-feature-vector/versions/1"

In [5]:
# Set up a model
model_1 = tf.keras.models.Sequential([
    hub.KerasLayer(mobilenet_v2, trainable = False, input_shape = (224, 224, 3)),
    tf.keras.layers.Dense(10, activation = 'softmax')
])

# Compile a model
model_1.compile(loss='categorical_crossentropy',
                optimizer='adam',
                metrics=['acc'])

# Train a model
history_1 = model_1.fit(x=train_data,
                        epochs=5,
                        validation_data=test_data)

Epoch 1/5
24/24 [==============================] - 24s 568ms/step - loss: 1.8591 - acc: 0.3560 - val_loss: 1.2395 - val_acc: 0.6392
Epoch 2/5
24/24 [==============================] - 9s 392ms/step - loss: 1.0039 - acc: 0.7187 - val_loss: 0.8717 - val_acc: 0.7512
Epoch 3/5
24/24 [==============================] - 10s 410ms/step - loss: 0.7259 - acc: 0.8093 - val_loss: 0.7428 - val_acc: 0.7660
Epoch 4/5
24/24 [==============================] - 10s 431ms/step - loss: 0.5779 - acc: 0.8507 - val_loss: 0.6699 - val_acc: 0.7984
Epoch 5/5
24/24 [==============================] - 12s 534ms/step - loss: 0.4805 - acc: 0.8813 - val_loss: 0.6316 - val_acc: 0.8060


It performs slightly better than the resnet model but worse than the efficientnet model.

## 2. Name 3 different image classification models on TensorFlow Hub that we haven't used.

- Inception
- Swin transformer
- Vision transformer

## 3. Build a model to classify images of two different things you've taken photos of.
- You can use any feature extraction layer from TensorFlow Hub you like for this.
- You should aim to have at least 10 images of each class, for example, to build a fridge versus oven classifier, you'll want 10 images of fridges and 10 images of ovens.

### Data preprocessing

To practice another option of handling image data, you can use a dataset from tensorflow and preprocess data instead of using `ImageDataGenerator`

In [6]:
import tensorflow_datasets as tfds

In [7]:
# Load data
(train_ds, test_ds), ds_info = tfds.load(name='cats_vs_dogs',
                                         data_dir='cvd',
                                         as_supervised=True,
                                         with_info=True,
                                         # way to split data if the dataset only has train data (80% as train and rest as test)
                                         split=['train[:80%]', 'train[80%:]'])

train_ds, test_ds

Dl Completed...: 0 url [00:00, ? url/s]

Dl Size...: 0 MiB [00:00, ? MiB/s]

Generating splits...:   0%|          | 0/1 [00:00<?, ? splits/s]

Generating train examples...:   0%|          | 0/23262 [00:00<?, ? examples/s]

Shuffling cvd/cats_vs_dogs/4.0.0.incompleteE5U1W2/cats_vs_dogs-train.tfrecord*...:   0%|          | 0/23262 [0…

Dataset cats_vs_dogs downloaded and prepared to cvd/cats_vs_dogs/4.0.0. Subsequent calls will reuse this data.


(<_PrefetchDataset element_spec=(TensorSpec(shape=(None, None, 3), dtype=tf.uint8, name=None), TensorSpec(shape=(), dtype=tf.int64, name=None))>,
 <_PrefetchDataset element_spec=(TensorSpec(shape=(None, None, 3), dtype=tf.uint8, name=None), TensorSpec(shape=(), dtype=tf.int64, name=None))>)

In [8]:
ds_info

tfds.core.DatasetInfo(
    name='cats_vs_dogs',
    full_name='cats_vs_dogs/4.0.0',
    description="""
    A large set of images of cats and dogs. There are 1738 corrupted images that are dropped.
    """,
    homepage='https://www.microsoft.com/en-us/download/details.aspx?id=54765',
    data_dir=PosixGPath('/tmp/tmpuq08j3k0tfds'),
    file_format=tfrecord,
    download_size=786.67 MiB,
    dataset_size=689.64 MiB,
    features=FeaturesDict({
        'image': Image(shape=(None, None, 3), dtype=uint8),
        'image/filename': Text(shape=(), dtype=string),
        'label': ClassLabel(shape=(), dtype=int64, num_classes=2),
    }),
    supervised_keys=('image', 'label'),
    disable_shuffling=False,
    splits={
        'train': <SplitInfo num_examples=23262, num_shards=8>,
    },
    citation="""@Inproceedings (Conference){asirra-a-captcha-that-exploits-interest-aligned-manual-image-categorization,
    author = {Elson, Jeremy and Douceur, John (JD) and Howell, Jon and Saul, Jared},
   

In [9]:
def preprocess_image(x, y):
    x /= 255
    x = tf.image.resize(x, size = (224, 224))
    return x, y

In [10]:
# preprocess data
train_ds = train_ds.map(preprocess_image)
test_ds = test_ds.map(preprocess_image)

train_ds, test_ds

(<_MapDataset element_spec=(TensorSpec(shape=(224, 224, 3), dtype=tf.float32, name=None), TensorSpec(shape=(), dtype=tf.int64, name=None))>,
 <_MapDataset element_spec=(TensorSpec(shape=(224, 224, 3), dtype=tf.float32, name=None), TensorSpec(shape=(), dtype=tf.int64, name=None))>)

In [11]:
# data into batches
train_ds = train_ds.batch(batch_size = 32)
test_ds = test_ds.batch(batch_size = 32)

train_ds, test_ds

(<_BatchDataset element_spec=(TensorSpec(shape=(None, 224, 224, 3), dtype=tf.float32, name=None), TensorSpec(shape=(None,), dtype=tf.int64, name=None))>,
 <_BatchDataset element_spec=(TensorSpec(shape=(None, 224, 224, 3), dtype=tf.float32, name=None), TensorSpec(shape=(None,), dtype=tf.int64, name=None))>)

### Model

In [12]:
# URL for the model
effnet_v2 = "https://www.kaggle.com/models/google/efficientnet-v2/frameworks/TensorFlow2/variations/imagenet21k-ft1k-b0-feature-vector/versions/1"

In [13]:
# Set up a model
model_2 = tf.keras.Sequential([
    hub.KerasLayer(handle = effnet_v2, trainable = False, input_shape = (224, 224, 3)),
    tf.keras.layers.Dense(1, activation = 'sigmoid')
])

# Compile a model
model_2.compile(optimizer='adam',
                loss='binary_crossentropy',
                metrics=['acc'])

# Train a model
history_2 = model_2.fit(x=train_ds,
                        epochs=5,
                        validation_data=test_ds)

Epoch 1/5
582/582 [==============================] - 54s 77ms/step - loss: 0.0611 - acc: 0.9796 - val_loss: 0.0276 - val_acc: 0.9914
Epoch 2/5
582/582 [==============================] - 44s 76ms/step - loss: 0.0249 - acc: 0.9924 - val_loss: 0.0244 - val_acc: 0.9920
Epoch 3/5
582/582 [==============================] - 45s 76ms/step - loss: 0.0208 - acc: 0.9935 - val_loss: 0.0235 - val_acc: 0.9925
Epoch 4/5
582/582 [==============================] - 44s 75ms/step - loss: 0.0180 - acc: 0.9944 - val_loss: 0.0232 - val_acc: 0.9927
Epoch 5/5
582/582 [==============================] - 43s 74ms/step - loss: 0.0157 - acc: 0.9951 - val_loss: 0.0232 - val_acc: 0.9927


## 4. What is the current best-performing model on ImageNet?
- Hint: you might want to check [sotabench.com](https://www.sotabench.com/) for this.

# 📖 Extra-curriculum
- Read through the [TensorFlow Transfer Learning Guide](https://www.tensorflow.org/tutorials/images/transfer_learning) and define the main two types of transfer learning in your own words.
- Go through the [Transfer Learning with TensorFlow Hub tutorial](https://www.tensorflow.org/tutorials/images/transfer_learning_with_hub) on the TensorFlow website and rewrite all of the code yourself into a new Google Colab notebook making comments about what each step does along the way.
- We haven't covered fine-tuning with TensorFlow Hub in this notebook, but if you'd like to know more, go through the [fine-tuning a TensorFlow Hub model tutorial on the TensorFlow homepage](https://www.tensorflow.org/hub/tf2_saved_model#fine-tuning). How to fine-tune a tensorflow hub model.
- Look into [experiment tracking with Weights & Biases](https://www.wandb.com/experiment-tracking), how could you integrate it with our existing TensorBoard logs?